<a href="https://colab.research.google.com/github/Sebastian-Frey/Timeseries-Forecasting-leveraging-LLMs/blob/main/6_Method3/chronos_geo_augmented_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data and model preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from country_matcher import add_location_columns
from tqdm.auto import tqdm

df_parquet = pd.read_parquet("weekly_aggregated_by_week_keyword.parquet")

# normalize to lowercase for a safe match
df_parquet["keyword"] = df_parquet["keyword"].astype(str).str.strip()
excluded_set = {k.lower() for k in EXCLUDED_KEYWORDS}

df_parquet = df_parquet[
    ~df_parquet["keyword"].str.lower().isin(excluded_set)
].copy()

# ============================
# 1) CONVERT WEEK -> ISO DATE
# ============================

df_parquet["week"] = df_parquet["week"].astype(str).str.strip()

# Convert "WW-YYYY" → "YYYYWW"
df_parquet["week_iso"] = df_parquet["week"].apply(
    lambda w: w.split("-")[1] + w.split("-")[0]
)

# Fix week 00 if present
df_parquet["week_iso"] = df_parquet["week_iso"].apply(
    lambda w: w[:-2] + "01" if w.endswith("00") else w
)

# Add Monday and parse to datetime
df_parquet["week_str"] = df_parquet["week_iso"] + "-1"
df_parquet["date"] = pd.to_datetime(df_parquet["week_str"], format="%G%V-%u", errors="coerce")

df_parquet = df_parquet.dropna(subset=["date"]).copy()

# ============================
# 2) AVERAGE DUPLICATES per (keyword, date)
# ============================

num_cols = df_parquet.select_dtypes(include="number").columns.tolist()

df_parquet = (
    df_parquet.groupby(["keyword", "date"], as_index=False)[num_cols]
      .mean()
)

# ============================
# 3) GLOBAL TIME WINDOW
# ============================

global_min = df_parquet["date"].min()
global_max = df_parquet["date"].max()

full_idx = pd.date_range(start=global_min, end=global_max, freq="W-MON")
print("GLOBAL TIME WINDOW:", global_min, "→", global_max)

# ============================
# 4) REINDEX EACH KEYWORD TO FULL WINDOW
# ============================

def reindex_one_keyword(g):
    g = g.set_index("date").reindex(full_idx)
    g.index.name = "date"
    g["keyword"] = g["keyword"].iloc[0]  # restore keyword column
    return g.reset_index()

df_parquet_full = (
    df_parquet.groupby("keyword", group_keys=False)
      .apply(reindex_one_keyword)
      .reset_index(drop=True)
)

# ============================
# 5) FILL NaNs USING KEYWORD MEAN
# ============================

for c in num_cols:
    df_parquet_full[c] = df_parquet_full[c].fillna(
        df_parquet_full.groupby("keyword")[c].transform("mean")
    )

print("FINAL SHAPE:", df_parquet_full.shape)
df_parquet_full.head()

# ============================
# 5b) ADD GEO COLUMNS WITH COUNTRY_MATCHER + PROGRESS BAR
# ============================

all_keywords = df_parquet_full["keyword"].dropna().unique().tolist()
total_kws = len(all_keywords)
print(f"Enriching {total_kws} keywords with geo info...")

pbar = tqdm(total=total_kws)

def progress_cb(current, total):
    # current and total come from add_location_columns
    # we just sync them to the tqdm bar
    pbar.n = current
    pbar.refresh()

df_parquet_full = add_location_columns(
    df_parquet_full,
    keywords=all_keywords,
    progress_callback=progress_cb,
    batch_size=10,   # callback every 10 keywords (tweak if you want)
)

pbar.close()

print("\nAdded geo columns:")
print(df_parquet_full[["keyword", "detected_city", "detected_country", "detected_continent"]].head())

# ============================
# 5c) ENCODE GEO COLUMNS AS NUMERIC IDS
# ============================

city_values       = df_parquet_full["detected_city"].dropna().unique()
country_values    = df_parquet_full["detected_country"].dropna().unique()
continent_values  = df_parquet_full["detected_continent"].dropna().unique()

city_to_id = {name: i for i, name in enumerate(sorted(city_values), start=1)}
country_to_id = {name: i for i, name in enumerate(sorted(country_values), start=1)}
continent_to_id = {name: i for i, name in enumerate(sorted(continent_values), start=1)}

df_parquet_full["detected_city_id"] = (
    df_parquet_full["detected_city"].map(city_to_id).fillna(0).astype("int32")
)
df_parquet_full["detected_country_id"] = (
    df_parquet_full["detected_country"].map(country_to_id).fillna(0).astype("int32")
)
df_parquet_full["detected_continent_id"] = (
    df_parquet_full["detected_continent"].map(continent_to_id).fillna(0).astype("int32")
)

print("\nSample with string geo columns:")
print(
    df_parquet_full[
        ["keyword", "detected_city", "detected_country", "detected_continent"]
    ].head().to_string(index=False)
)

print("\nSample with numeric geo IDs:")
print(
    df_parquet_full[
        ["keyword", "detected_city_id", "detected_country_id", "detected_continent_id"]
    ].head().to_string(index=False)
)

# ============================
# 6) CLEAN NaN / EMPTY KEYWORDS BEFORE CHRONOS
# ============================

print("Rows in df_parquet_full BEFORE cleaning:", len(df_parquet_full))
print("NaN in keyword BEFORE:", df_parquet_full["keyword"].isna().sum())

# 1) drop rows with keyword = NaN
df_parquet_full = df_parquet_full[df_parquet_full["keyword"].notna()].copy()

# 2) drop rows with keyword = "" or only spaces
df_parquet_full = df_parquet_full[
    df_parquet_full["keyword"].astype(str).str.strip() != ""
].copy()

# 3) (optional) make sure date and target are non-null too
df_parquet_full = df_parquet_full[df_parquet_full["date"].notna()].copy()
df_parquet_full = df_parquet_full[df_parquet_full["cpc_week"].notna()].copy()

print("Rows in df_parquet_full AFTER cleaning:", len(df_parquet_full))
print("NaN in keyword AFTER:", df_parquet_full["keyword"].isna().sum())
print("Distinct keywords AFTER:", df_parquet_full["keyword"].nunique())

# ============================
# 7) KEYWORD INSPECTION TOOL
# ============================

def inspect_keyword(keyword, plot_col="cpc_week"):
    df_parquet_k = df_parquet_full[df_parquet_full["keyword"].str.lower() == keyword.lower()].copy()

    if df_parquet_k.empty:
        print(f"Keyword not found: {keyword}")
        return df_parquet_k

    print("\n==============================")
    print("     INSPECTION:", keyword)
    print("==============================")
    print("Rows:", len(df_parquet_k))
    print("Date range:", df_parquet_k["date"].min(), "→", df_parquet_k["date"].max())
    print("\nNulls per column:")
    print(df_parquet_k.isna().sum())

    if plot_col in df_parquet_k.columns:
        print(f"\nMean value used for '{plot_col}' imputation:",
              df_parquet_k[plot_col].mean())

    print("\nHEAD (first 10 rows):")
    print(df_parquet_k.head(10).to_string())

    # plot
    if plot_col in df_parquet_k.columns:
        plt.figure(figsize=(12,4))
        plt.plot(df_parquet_k["date"], df_parquet_k[plot_col])
        plt.title(f"{keyword} – {plot_col} over time")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

    return df_parquet_k

# ============================
# 7) EXAMPLE
# ============================

inspect_keyword("24 hour rent")


In [ ]:
# ============================
# 7) PANEL FOR CHRONOS (ALL KEYWORDS)
# ============================

df_kw_loop = df_parquet_full.copy()

print("Total rows in df_kw_loop (all keywords):", len(df_kw_loop))
print("Total distinct keywords:", df_kw_loop["keyword"].nunique())

all_keywords_full = sorted(
    df_kw_loop["keyword"]
    .astype(str)
    .str.strip()
    .unique()
)
print(f"Total keywords in full panel: {len(all_keywords_full)}")


In [ ]:
import torch
from chronos import Chronos2Pipeline

device_map = "cuda" if torch.cuda.is_available() else "cpu"

pipeline = Chronos2Pipeline.from_pretrained(
    "s3://autogluon/chronos-2",
    device_map=device_map,
)


## Defining the keywords to plot

## Zero-shot model

### Without geo-variables

In [ ]:
import torch
from chronos import Chronos2Pipeline

# =========================================
# 8) ZERO-SHOT BASELINE OVER MULTIPLE HORIZONS
# =========================================

ID_COL     = "keyword"
TIME_COL   = "date"
TARGET_COL = "cpc_week"

PRED_LENGTHS = [1, 6, 12]
metrics_zero_by_horizon = {}

# ---- PLOT CONFIG (you will fill KEYWORDS_TO_PLOT later) ----
N_PLOT_DEFAULT   = 3
RANDOM_SEED      = 42

def get_keywords_to_plot(all_keywords, keywords_to_plot=None, n_plot_default=3, seed=42):
    all_keywords = [str(k).strip() for k in all_keywords if pd.notna(k)]
    all_set = {k.lower() for k in all_keywords}

    # user-provided list
    if keywords_to_plot is not None and len(keywords_to_plot) > 0:
        chosen, missing = [], []
        for k in keywords_to_plot:
            kk = str(k).strip()
            if kk.lower() in all_set:
                chosen.append(kk)
            else:
                missing.append(kk)

        if missing:
            print("⚠️ Requested plot keywords not found in test-set keywords (skipped):")
            for m in missing:
                print(" -", m)

        return chosen

    # fallback random sample
    if len(all_keywords) == 0:
        return []

    np.random.seed(seed)
    n_plot = min(n_plot_default, len(all_keywords))
    return list(np.random.choice(all_keywords, size=n_plot, replace=False))


for PRED_LEN in PRED_LENGTHS:
    print("\n" + "="*70)
    print(f"🔵 ZERO-SHOT CHRONOS-2 – PREDICTION HORIZON = {PRED_LEN} weeks")
    print("="*70)

    # ============================
    # 9) GLOBAL TRAIN / TEST SPLIT
    # ============================

    max_date  = df_kw_loop[TIME_COL].max()
    train_end = max_date - pd.Timedelta(weeks=PRED_LEN)

    train_panel = df_kw_loop[df_kw_loop[TIME_COL] <= train_end].copy()
    test_panel  = df_kw_loop[df_kw_loop[TIME_COL] >  train_end].copy()

    # 👇 EXCLUDE GEO VARIABLES ONLY IN THIS ZERO-SHOT SETUP
    cols_to_drop = [
        "detected_city_id",
        "detected_country_id",
        "detected_continent_id",
    ]

    train_panel = train_panel.drop(columns=cols_to_drop, errors="ignore")
    test_panel  = test_panel.drop(columns=cols_to_drop, errors="ignore")

    # ensure keyword is consistently string (prevents sorted/type issues)
    train_panel[ID_COL] = train_panel[ID_COL].astype(str).str.strip()
    test_panel[ID_COL]  = test_panel[ID_COL].astype(str).str.strip()

    print("Train range:", train_panel[TIME_COL].min(), "→", train_panel[TIME_COL].max())
    print("Test  range:",  test_panel[TIME_COL].min(),  "→", test_panel[TIME_COL].max())

    # ============================
    # 10) ZERO-SHOT FORECAST WITH BASE MODEL
    # ============================

    pred_df_zero = pipeline.predict_df(
        train_panel,
        prediction_length=PRED_LEN,
        quantile_levels=[0.1, 0.5, 0.9],
        id_column=ID_COL,
        timestamp_column=TIME_COL,
        target=TARGET_COL,
    )

    # Keep only forecast horizon
    pred_df_zero = pred_df_zero[pred_df_zero[TIME_COL] > train_end].copy()

    print("Zero-shot prediction sample:")
    print(pred_df_zero.head().to_string(index=False))

    # ============================
    # 11) METRICS PER KEYWORD (ZERO-SHOT)
    #     (computed over ALL keywords that appear in the test set)
    # ============================

    all_keywords = (
        test_panel[ID_COL]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
        .tolist()
    )

    records = []

    for kw in all_keywords:
        # actuals in test period
        test_kw = test_panel[test_panel[ID_COL].str.lower() == kw.lower()].copy()
        if test_kw.empty:
            continue

        # predictions for this keyword
        pred_kw = pred_df_zero[pred_df_zero[ID_COL].str.lower() == kw.lower()].copy()
        pred_kw = pred_kw.sort_values(TIME_COL)

        # align by date
        merged = test_kw[[TIME_COL, TARGET_COL]].merge(
            pred_kw[[TIME_COL, "0.5"]],
            on=TIME_COL,
            how="inner",
        )

        if merged.empty:
            continue

        y_true = merged[TARGET_COL].values
        y_pred = merged["0.5"].values

        # MAE
        mae  = np.mean(np.abs(y_true - y_pred))

        # MAPE (as fraction, not %)
        denom = np.where(y_true == 0, 1e-6, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / denom))

        # sMAPE (also as fraction, not %)
        denom_smape = np.abs(y_true) + np.abs(y_pred)
        denom_smape = np.where(denom_smape == 0, 1e-6, denom_smape)
        smape = np.mean(2.0 * np.abs(y_true - y_pred) / denom_smape)

        # RMSE
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

        records.append({
            "keyword": kw,
            "mae": mae,
            "mape": mape,
            "smape": smape,
            "rmse": rmse,
        })


    metrics_zero = pd.DataFrame(records).sort_values("mae").reset_index(drop=True)
    metrics_zero_by_horizon[PRED_LEN] = metrics_zero

    print(f"\n=== ZERO-SHOT MODEL – METRICS PER KEYWORD (horizon={PRED_LEN}) ===")
    print(metrics_zero.head(20).to_string(index=False))

    # === ZERO-SHOT MODEL – AGGREGATED METRICS (MEAN ± STD) ===

    mean_smape_zero = metrics_zero["smape"].mean()
    std_smape_zero  = metrics_zero["smape"].std()

    mean_rmse_zero = metrics_zero["rmse"].mean()
    std_rmse_zero  = metrics_zero["rmse"].std()

    print(f"\n=== ZERO-SHOT MODEL – AGGREGATED METRICS (horizon={PRED_LEN}) ===")
    print(f"sMAPE (mean ± std) : {mean_smape_zero:.4f} ± {std_smape_zero:.4f}")
    print(f"RMSE  (mean ± std) : {mean_rmse_zero:.4f} ± {std_rmse_zero:.4f}")

    # ============================
    # 12) PLOT FUNCTION FOR ZERO-SHOT MODEL (THIS HORIZON)
    # ============================

    def plot_zero_shot_forecast(keyword):
        test_kw = test_panel[test_panel[ID_COL].str.lower() == keyword.lower()].copy()
        if test_kw.empty:
            print(f"No test data for keyword: {keyword}")
            return

        kw_all = df_kw_loop[df_kw_loop[ID_COL].str.lower() == keyword.lower()].copy()
        kw_all = kw_all.sort_values(TIME_COL)

        pred_kw = pred_df_zero[pred_df_zero[ID_COL].str.lower() == keyword.lower()].copy()
        pred_kw = pred_kw.sort_values(TIME_COL)

        if pred_kw.empty:
            print(f"No predictions for keyword: {keyword}")
            return

        plt.figure(figsize=(12, 4))

        # full actual history
        plt.plot(
            kw_all[TIME_COL],
            kw_all[TARGET_COL],
            "o-",
            color="black",
            markersize=3,
            linewidth=1,
            label="Actual (full history)",
        )

        # forecast median
        plt.plot(
            pred_kw[TIME_COL],
            pred_kw["0.5"],
            "s--",
            color="blue",
            linewidth=1.8,
            markersize=5,
            label=f"Zero-shot forecast (median, horizon={PRED_LEN})",
        )

        # prediction interval
        plt.fill_between(
            pred_kw[TIME_COL],
            pred_kw["0.1"],
            pred_kw["0.9"],
            alpha=0.25,
            label="P10–P90 interval",
        )

        # test actuals
        plt.scatter(
            test_kw[TIME_COL],
            test_kw[TARGET_COL],
            color="red",
            marker="D",
            s=50,
            label="Test set (actual)",
            zorder=5,
        )

        # train/test split
        plt.axvline(
            train_end,
            linestyle="--",
            color="gray",
            alpha=0.7,
            label="Train/Test split",
        )

        plt.title(f"Zero-shot no geo-variables | {keyword} (horizon={PRED_LEN})", fontsize=14)
        plt.xlabel("Week", fontsize=11)
        plt.ylabel("Average CPC", fontsize=11)
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend(fontsize=9)
        plt.tight_layout()
        plt.show()

    # ============================
    # 13) PLOT SELECTED (OR RANDOM) KEYWORDS FOR THIS HORIZON
    # ============================

    keywords_to_plot = get_keywords_to_plot(
        all_keywords=all_keywords,
        keywords_to_plot=KEYWORDS_TO_PLOT,   # <-- you'll set later
        n_plot_default=N_PLOT_DEFAULT,
        seed=RANDOM_SEED
    )

    print(f"\nPlotting {len(keywords_to_plot)} keywords (zero-shot, horizon={PRED_LEN}):")
    for kw in keywords_to_plot:
        print(" -", kw)
        plot_zero_shot_forecast(kw)


### With geo-variables

In [ ]:
import torch
from chronos import Chronos2Pipeline

# =========================================
# 8) ZERO-SHOT BASELINE OVER MULTIPLE HORIZONS
# =========================================

ID_COL     = "keyword"
TIME_COL   = "date"
TARGET_COL = "cpc_week"

PRED_LENGTHS = [1, 6, 12]
metrics_zero_by_horizon = {}

def get_keywords_to_plot(all_keywords, keywords_to_plot=None, n_plot_default=3, seed=42):
    all_keywords = [str(k).strip() for k in all_keywords if pd.notna(k)]
    all_set = {k.lower() for k in all_keywords}

    # user list
    if keywords_to_plot is not None and len(keywords_to_plot) > 0:
        chosen, missing = [], []
        for k in keywords_to_plot:
            kk = str(k).strip()
            if kk.lower() in all_set:
                chosen.append(kk)
            else:
                missing.append(kk)

        if missing:
            print("⚠️ Plot keywords NOT found in this horizon test-set keyword list (skipped):")
            for m in missing:
                print(" -", m)

        return chosen

    # fallback random
    if len(all_keywords) == 0:
        return []

    np.random.seed(seed)
    n_plot = min(n_plot_default, len(all_keywords))
    return list(np.random.choice(all_keywords, size=n_plot, replace=False))


for PRED_LEN in PRED_LENGTHS:
    print("\n" + "="*70)
    print(f"🔵 ZERO-SHOT CHRONOS-2 – PREDICTION HORIZON = {PRED_LEN} weeks")
    print("="*70)

    # ============================
    # 9) GLOBAL TRAIN / TEST SPLIT
    # ============================

    max_date  = df_kw_loop[TIME_COL].max()
    train_end = max_date - pd.Timedelta(weeks=PRED_LEN)

    train_panel = df_kw_loop[df_kw_loop[TIME_COL] <= train_end].copy()
    test_panel  = df_kw_loop[df_kw_loop[TIME_COL] >  train_end].copy()

    # 👇 EXCLUDE GEO VARIABLES ONLY IN THIS ZERO-SHOT SETUP (if you want)
    cols_to_drop = []
    train_panel = train_panel.drop(columns=cols_to_drop, errors="ignore")
    test_panel  = test_panel.drop(columns=cols_to_drop, errors="ignore")

    # ensure keyword is clean string (prevents float vs str issues)
    train_panel[ID_COL] = train_panel[ID_COL].astype(str).str.strip()
    test_panel[ID_COL]  = test_panel[ID_COL].astype(str).str.strip()

    print("Train range:", train_panel[TIME_COL].min(), "→", train_panel[TIME_COL].max())
    print("Test  range:",  test_panel[TIME_COL].min(),  "→", test_panel[TIME_COL].max())

    # ============================
    # 10) ZERO-SHOT FORECAST WITH BASE MODEL
    # ============================

    pred_df_zero = pipeline.predict_df(
        train_panel,
        prediction_length=PRED_LEN,
        quantile_levels=[0.1, 0.5, 0.9],
        id_column=ID_COL,
        timestamp_column=TIME_COL,
        target=TARGET_COL,
    )

    # Keep only forecast horizon
    pred_df_zero = pred_df_zero[pred_df_zero[TIME_COL] > train_end].copy()

    print("Zero-shot prediction sample:")
    print(pred_df_zero.head().to_string(index=False))

    # ============================
    # 11) METRICS PER KEYWORD (ZERO-SHOT)
    #    computed over ALL keywords that appear in THIS test_panel
    # ============================

    all_keywords = (
        test_panel[ID_COL]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
        .tolist()
    )

    records = []

    for kw in all_keywords:
        # actuals in test period
        test_kw = test_panel[test_panel[ID_COL].str.lower() == kw.lower()].copy()
        if test_kw.empty:
            continue

        # predictions for this keyword
        pred_kw = pred_df_zero[pred_df_zero[ID_COL].str.lower() == kw.lower()].copy()
        pred_kw = pred_kw.sort_values(TIME_COL)

        # align by date
        merged = test_kw[[TIME_COL, TARGET_COL]].merge(
            pred_kw[[TIME_COL, "0.5"]],
            on=TIME_COL,
            how="inner",
        )

        if merged.empty:
            continue

        y_true = merged[TARGET_COL].values
        y_pred = merged["0.5"].values

        mae  = np.mean(np.abs(y_true - y_pred))

        denom = np.where(y_true == 0, 1e-6, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / denom))

        denom_smape = np.abs(y_true) + np.abs(y_pred)
        denom_smape = np.where(denom_smape == 0, 1e-6, denom_smape)
        smape = np.mean(2.0 * np.abs(y_true - y_pred) / denom_smape)

        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

        records.append({
            "keyword": kw,
            "mae": mae,
            "mape": mape,
            "smape": smape,
            "rmse": rmse,
        })

    metrics_zero = pd.DataFrame(records).sort_values("mae").reset_index(drop=True)
    metrics_zero_by_horizon[PRED_LEN] = metrics_zero

    mean_smape = metrics_zero["smape"].mean()
    std_smape  = metrics_zero["smape"].std()
    mean_rmse  = metrics_zero["rmse"].mean()
    std_rmse   = metrics_zero["rmse"].std()

    print(f"\n=== ZERO-SHOT – AGGREGATED METRICS (horizon={PRED_LEN}) ===")
    print(f"sMAPE (mean ± std) : {mean_smape:.4f} ± {std_smape:.4f}")
    print(f"RMSE  (mean ± std) : {mean_rmse:.4f} ± {std_rmse:.4f}")

    # ============================
    # 12) PLOT FUNCTION FOR ZERO-SHOT MODEL (THIS HORIZON)
    # ============================

    def plot_zero_shot_forecast(keyword):
        test_kw = test_panel[test_panel[ID_COL].str.lower() == keyword.lower()].copy()
        if test_kw.empty:
            print(f"⚠️ No test data for keyword (h={PRED_LEN}): {keyword}")
            return

        kw_all = df_kw_loop[df_kw_loop[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)

        pred_kw = pred_df_zero[pred_df_zero[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)
        if pred_kw.empty:
            print(f"⚠️ No predictions for keyword (h={PRED_LEN}): {keyword}")
            return

        plt.figure(figsize=(12, 4))

        plt.plot(
            kw_all[TIME_COL],
            kw_all[TARGET_COL],
            "o-",
            color="black",
            markersize=3,
            linewidth=1,
            label="Actual (full history)",
        )

        plt.plot(
            pred_kw[TIME_COL],
            pred_kw["0.5"],
            "s--",
            color="blue",
            linewidth=1.8,
            markersize=5,
            label=f"Zero-shot forecast (median, horizon={PRED_LEN})",
        )

        plt.fill_between(
            pred_kw[TIME_COL],
            pred_kw["0.1"],
            pred_kw["0.9"],
            alpha=0.25,
            label="P10–P90 interval",
        )

        plt.scatter(
            test_kw[TIME_COL],
            test_kw[TARGET_COL],
            color="red",
            marker="D",
            s=50,
            label="Test set (actual)",
            zorder=5,
        )

        plt.axvline(
            train_end,
            linestyle="--",
            color="gray",
            alpha=0.7,
            label="Train/Test split",
        )

        plt.title(f"Chronos-2 Zero-shot with geo-variables | {keyword} (horizon={PRED_LEN})", fontsize=14)
        plt.xlabel("Week", fontsize=11)
        plt.ylabel("Average CPC", fontsize=11)
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend(fontsize=9)
        plt.tight_layout()
        plt.show()

    # ============================
    # 13) PLOT YOUR SELECTED KEYWORDS (OR RANDOM FALLBACK)
    # ============================

    keywords_to_plot = get_keywords_to_plot(
        all_keywords=all_keywords,          # use test-set keywords for this horizon
        keywords_to_plot=KEYWORDS_TO_PLOT,  # <-- your list
        n_plot_default=N_PLOT_DEFAULT,
        seed=RANDOM_SEED,
    )

    print(f"\nPlotting {len(keywords_to_plot)} keywords (zero-shot, horizon={PRED_LEN}):")
    for kw in keywords_to_plot:
        print(" -", kw)
        plot_zero_shot_forecast(kw)


## With covariates model

### Without geo-variables

In [ ]:
import torch
from chronos import Chronos2Pipeline

ID_COL     = "keyword"
TIME_COL   = "date"
TARGET_COL = "cpc_week"

# horizons you want to test
PRED_LENGTHS = [1, 6, 12]

# Option B: keep a default random sample size (used if KEYWORDS_TO_PLOT is None)
N_PLOT_DEFAULT = 3
RANDOM_SEED = 42

# to store metrics per horizon if you want later comparisons
metrics_by_horizon = {}

def get_keywords_to_plot(all_keywords, keywords_to_plot=None, n_plot_default=3, seed=42):
    all_keywords = [str(k).strip() for k in all_keywords if pd.notna(k)]
    all_set = {k.lower() for k in all_keywords}

    if keywords_to_plot is not None and len(keywords_to_plot) > 0:
        chosen, missing = [], []
        for k in keywords_to_plot:
            kk = str(k).strip()
            if kk.lower() in all_set:
                chosen.append(kk)
            else:
                missing.append(kk)

        if missing:
            print("⚠️ Plot keywords NOT found in this horizon test-set keyword list (skipped):")
            for m in missing:
                print(" -", m)

        return chosen

    if len(all_keywords) == 0:
        return []

    np.random.seed(seed)
    n_plot = min(n_plot_default, len(all_keywords))
    return list(np.random.choice(all_keywords, size=n_plot, replace=False))


for PRED_LEN in PRED_LENGTHS:
    print("\n" + "="*70)
    print(f"🔮 GLOBAL MODEL – PREDICTION HORIZON = {PRED_LEN} weeks")
    print("="*70)

    # ============================
    # 9) GLOBAL TRAIN / TEST SPLIT
    # ============================
    max_date  = df_kw_loop[TIME_COL].max()
    train_end = max_date - pd.Timedelta(weeks=PRED_LEN)

    train_panel = df_kw_loop[df_kw_loop[TIME_COL] <= train_end].copy()
    test_panel  = df_kw_loop[df_kw_loop[TIME_COL] >  train_end].copy()

    # ensure keyword is clean string (prevents float vs str issues)
    train_panel[ID_COL] = train_panel[ID_COL].astype(str).str.strip()
    test_panel[ID_COL]  = test_panel[ID_COL].astype(str).str.strip()

    print("Train range:", train_panel[TIME_COL].min(), "→", train_panel[TIME_COL].max())
    print("Test  range:",  test_panel[TIME_COL].min(),  "→", test_panel[TIME_COL].max())

    # ============================
    # 10) BUILD GLOBAL TRAIN INPUTS (NUMERIC COVARIATES)
    # ============================
    covariate_cols = [
        "impressions_sum",
        "n_st_branded_search",
        "n_st_generic_search",
        "n_dev_desktop",
        "n_dev_mobile",
        "n_dev_tablet",
        "adclicks_sum",
        "avg_sim_top25_this_week",
        "avg_sim_top25_last_week",
        "n_sim_this_week",
        "n_sim_last_week",
        "detected_city",
        "detected_country",
        "detected_continent"
    ]


    inputs = []

    for kw, g in train_panel.groupby(ID_COL):
        g = g.sort_values(TIME_COL)

        # ensure enough history
        if len(g) < PRED_LEN + 10:
            continue

        series = {
            "target": g[TARGET_COL].values.astype("float32"),
            "past_covariates": {},
            "future_covariates": {},
        }

        for col in covariate_cols:
            if col in g.columns and np.issubdtype(g[col].dtype, np.number):
                series["past_covariates"][col] = g[col].values.astype("float32")

        inputs.append(series)

    print(f"Prepared {len(inputs)} series for global training (horizon={PRED_LEN}).")

    if not inputs:
        print("❌ No series available for training at this horizon. Skipping.")
        continue

    # ============================
    # 11) GLOBAL FINE-TUNING
    # ============================
    finetuned_global = pipeline.fit(
        inputs=inputs,
        prediction_length=PRED_LEN,
        num_steps=50,
        learning_rate=1e-5,
        batch_size=32,
        logging_steps=10,
    )

    # ============================
    # 12) GLOBAL FORECAST + METRICS PER KEYWORD
    # ============================

    pred_df = finetuned_global.predict_df(
        train_panel,
        prediction_length=PRED_LEN,
        quantile_levels=[0.1, 0.5, 0.9],
        id_column=ID_COL,
        timestamp_column=TIME_COL,
        target=TARGET_COL,
    )

    pred_df = pred_df[pred_df[TIME_COL] > train_end].copy()

    print("Prediction sample:")
    print(pred_df.head().to_string(index=False))

    # keywords to evaluate = all keywords that appear in THIS horizon test set
    all_keywords_h = (
        test_panel[ID_COL]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
        .tolist()
    )

    records = []

    for kw in all_keywords_h:
        test_kw = test_panel[test_panel[ID_COL].str.lower() == kw.lower()].copy()
        if test_kw.empty:
            continue

        pred_kw = pred_df[pred_df[ID_COL].str.lower() == kw.lower()].copy().sort_values(TIME_COL)

        merged = test_kw[[TIME_COL, TARGET_COL]].merge(
            pred_kw[[TIME_COL, "0.5"]],
            on=TIME_COL,
            how="inner",
        )
        if merged.empty:
            continue

        y_true = merged[TARGET_COL].values
        y_pred = merged["0.5"].values

        mae  = np.mean(np.abs(y_true - y_pred))

        denom = np.where(y_true == 0, 1e-6, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / denom))

        denom_smape = np.abs(y_true) + np.abs(y_pred)
        denom_smape = np.where(denom_smape == 0, 1e-6, denom_smape)
        smape = np.mean(2.0 * np.abs(y_true - y_pred) / denom_smape)

        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

        records.append({"keyword": kw, "mae": mae, "mape": mape, "smape": smape, "rmse": rmse})

    metrics_global = pd.DataFrame(records).sort_values("mae").reset_index(drop=True)
    metrics_by_horizon[PRED_LEN] = metrics_global

    print(f"\n=== GLOBAL MODEL – METRICS PER KEYWORD (horizon={PRED_LEN}) ===")
    print(metrics_global.head(20).to_string(index=False))

    mean_smape = metrics_global["smape"].mean()
    std_smape  = metrics_global["smape"].std()
    mean_rmse  = metrics_global["rmse"].mean()
    std_rmse   = metrics_global["rmse"].std()

    print(f"\n=== GLOBAL MODEL – AGGREGATED METRICS (horizon={PRED_LEN}) ===")
    print(f"sMAPE (mean ± std) : {mean_smape:.4f} ± {std_smape:.4f}")
    print(f"RMSE  (mean ± std) : {mean_rmse:.4f} ± {std_rmse:.4f}")

    # ============================
    # 13) PLOT FUNCTION
    # ============================
    def plot_global_forecast(keyword):
        test_kw = test_panel[test_panel[ID_COL].str.lower() == keyword.lower()].copy()
        if test_kw.empty:
            print(f"⚠️ No test data for keyword (h={PRED_LEN}): {keyword}")
            return

        kw_all = df_kw_loop[df_kw_loop[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)

        pred_kw = pred_df[pred_df[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)
        if pred_kw.empty:
            print(f"⚠️ No predictions for keyword (h={PRED_LEN}): {keyword}")
            return

        plt.figure(figsize=(12, 4))

        plt.plot(
            kw_all[TIME_COL],
            kw_all[TARGET_COL],
            "o-",
            color="black",
            markersize=3,
            linewidth=1,
            label="Actual (full history)",
        )

        plt.plot(
            pred_kw[TIME_COL],
            pred_kw["0.5"],
            "s--",
            color="purple",
            linewidth=1.8,
            markersize=5,
            label="Forecast (median)",
        )

        plt.fill_between(
            pred_kw[TIME_COL],
            pred_kw["0.1"],
            pred_kw["0.9"],
            color="purple",
            alpha=0.25,
            label="P10–P90 interval",
        )

        plt.scatter(
            test_kw[TIME_COL],
            test_kw[TARGET_COL],
            color="red",
            marker="D",
            s=50,
            label="Test set (actual)",
            zorder=5,
        )

        plt.axvline(
            train_end,
            linestyle="--",
            color="gray",
            alpha=0.7,
            label="Train/Test split",
        )

        plt.title(f"Global Fine-Tuned Model without geo-variables | {keyword} (horizon={PRED_LEN})", fontsize=14)
        plt.xlabel("Week", fontsize=11)
        plt.ylabel("Average CPC", fontsize=11)
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend(fontsize=9)
        plt.tight_layout()
        plt.show()

    # ============================
    # 14) PLOT YOUR SELECTED KEYWORDS (OR RANDOM FALLBACK)
    # ============================
    keywords_to_plot = get_keywords_to_plot(
        all_keywords=all_keywords_h,         # test-set keywords for this horizon
        keywords_to_plot=KEYWORDS_TO_PLOT,   # <-- your list
        n_plot_default=N_PLOT_DEFAULT,
        seed=RANDOM_SEED,
    )

    print(f"\nPlotting {len(keywords_to_plot)} keywords (global, horizon={PRED_LEN}):")
    for kw in keywords_to_plot:
        print(" -", kw)
        plot_global_forecast(kw)


### With geo-variables

In [ ]:
import torch
from chronos import Chronos2Pipeline

# Option B: keep a default random sample size (used if KEYWORDS_TO_PLOT is None)
N_PLOT_DEFAULT = 3
RANDOM_SEED = 42

ID_COL     = "keyword"
TIME_COL   = "date"
TARGET_COL = "cpc_week"

# horizons you want to test
PRED_LENGTHS = [1, 6, 12]

# to store metrics per horizon if you want later comparisons
metrics_by_horizon = {}

def get_keywords_to_plot(all_keywords, keywords_to_plot=None, n_plot_default=3, seed=42):
    all_keywords = [str(k).strip() for k in all_keywords if pd.notna(k)]
    all_set = {k.lower() for k in all_keywords}

    if keywords_to_plot is not None and len(keywords_to_plot) > 0:
        chosen, missing = [], []
        for k in keywords_to_plot:
            kk = str(k).strip()
            if kk.lower() in all_set:
                chosen.append(kk)
            else:
                missing.append(kk)

        if missing:
            print("⚠️ Plot keywords NOT found in this horizon test-set keyword list (skipped):")
            for m in missing:
                print(" -", m)

        return chosen

    if len(all_keywords) == 0:
        return []

    np.random.seed(seed)
    n_plot = min(n_plot_default, len(all_keywords))
    return list(np.random.choice(all_keywords, size=n_plot, replace=False))


for PRED_LEN in PRED_LENGTHS:
    print("\n" + "="*70)
    print(f"🔮 GLOBAL MODEL – PREDICTION HORIZON = {PRED_LEN} weeks")
    print("="*70)

    # ============================
    # 9) GLOBAL TRAIN / TEST SPLIT
    # ============================
    max_date  = df_kw_loop[TIME_COL].max()
    train_end = max_date - pd.Timedelta(weeks=PRED_LEN)

    train_panel = df_kw_loop[df_kw_loop[TIME_COL] <= train_end].copy()
    test_panel  = df_kw_loop[df_kw_loop[TIME_COL] >  train_end].copy()

    # ensure keyword is clean string (prevents float vs str issues)
    train_panel[ID_COL] = train_panel[ID_COL].astype(str).str.strip()
    test_panel[ID_COL]  = test_panel[ID_COL].astype(str).str.strip()

    print("Train range:", train_panel[TIME_COL].min(), "→", train_panel[TIME_COL].max())
    print("Test  range:",  test_panel[TIME_COL].min(),  "→", test_panel[TIME_COL].max())

    # ============================
    # 10) BUILD GLOBAL TRAIN INPUTS (NUMERIC COVARIATES)
    # ============================
    covariate_cols = [
        "impressions_sum",
        "n_st_branded_search",
        "n_st_generic_search",
        "n_dev_desktop",
        "n_dev_mobile",
        "n_dev_tablet",
        "adclicks_sum",
        "avg_sim_top25_this_week",
        "avg_sim_top25_last_week",
        "n_sim_this_week",
        "n_sim_last_week",
        "detected_city_id",
        "detected_country_id",
        "detected_continent_id"
    ]


    inputs = []

    for kw, g in train_panel.groupby(ID_COL):
        g = g.sort_values(TIME_COL)

        # ensure enough history
        if len(g) < PRED_LEN + 10:
            continue

        series = {
            "target": g[TARGET_COL].values.astype("float32"),
            "past_covariates": {},
            "future_covariates": {},
        }

        for col in covariate_cols:
            if col in g.columns and np.issubdtype(g[col].dtype, np.number):
                series["past_covariates"][col] = g[col].values.astype("float32")

        inputs.append(series)

    print(f"Prepared {len(inputs)} series for global training (horizon={PRED_LEN}).")

    if not inputs:
        print("❌ No series available for training at this horizon. Skipping.")
        continue

    # ============================
    # 11) GLOBAL FINE-TUNING
    # ============================
    finetuned_global = pipeline.fit(
        inputs=inputs,
        prediction_length=PRED_LEN,
        num_steps=50,
        learning_rate=1e-5,
        batch_size=32,
        logging_steps=10,
    )

    # ============================
    # 12) GLOBAL FORECAST + METRICS PER KEYWORD
    # ============================

    pred_df = finetuned_global.predict_df(
        train_panel,
        prediction_length=PRED_LEN,
        quantile_levels=[0.1, 0.5, 0.9],
        id_column=ID_COL,
        timestamp_column=TIME_COL,
        target=TARGET_COL,
    )

    pred_df = pred_df[pred_df[TIME_COL] > train_end].copy()

    print("Prediction sample:")
    print(pred_df.head().to_string(index=False))

    # keywords to evaluate = all keywords that appear in THIS horizon test set
    all_keywords_h = (
        test_panel[ID_COL]
        .dropna()
        .astype(str)
        .str.strip()
        .unique()
        .tolist()
    )

    records = []

    for kw in all_keywords_h:
        test_kw = test_panel[test_panel[ID_COL].str.lower() == kw.lower()].copy()
        if test_kw.empty:
            continue

        pred_kw = pred_df[pred_df[ID_COL].str.lower() == kw.lower()].copy().sort_values(TIME_COL)

        merged = test_kw[[TIME_COL, TARGET_COL]].merge(
            pred_kw[[TIME_COL, "0.5"]],
            on=TIME_COL,
            how="inner",
        )
        if merged.empty:
            continue

        y_true = merged[TARGET_COL].values
        y_pred = merged["0.5"].values

        mae  = np.mean(np.abs(y_true - y_pred))

        denom = np.where(y_true == 0, 1e-6, y_true)
        mape = np.mean(np.abs((y_true - y_pred) / denom))

        denom_smape = np.abs(y_true) + np.abs(y_pred)
        denom_smape = np.where(denom_smape == 0, 1e-6, denom_smape)
        smape = np.mean(2.0 * np.abs(y_true - y_pred) / denom_smape)

        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

        records.append({"keyword": kw, "mae": mae, "mape": mape, "smape": smape, "rmse": rmse})

    metrics_global = pd.DataFrame(records).sort_values("mae").reset_index(drop=True)
    metrics_by_horizon[PRED_LEN] = metrics_global

    print(f"\n=== GLOBAL MODEL – METRICS PER KEYWORD (horizon={PRED_LEN}) ===")
    print(metrics_global.head(20).to_string(index=False))

    mean_smape = metrics_global["smape"].mean()
    std_smape  = metrics_global["smape"].std()
    mean_rmse  = metrics_global["rmse"].mean()
    std_rmse   = metrics_global["rmse"].std()

    print(f"\n=== GLOBAL MODEL – AGGREGATED METRICS (horizon={PRED_LEN}) ===")
    print(f"sMAPE (mean ± std) : {mean_smape:.4f} ± {std_smape:.4f}")
    print(f"RMSE  (mean ± std) : {mean_rmse:.4f} ± {std_rmse:.4f}")

    # ============================
    # 13) PLOT FUNCTION
    # ============================
    def plot_global_forecast(keyword):
        test_kw = test_panel[test_panel[ID_COL].str.lower() == keyword.lower()].copy()
        if test_kw.empty:
            print(f"⚠️ No test data for keyword (h={PRED_LEN}): {keyword}")
            return

        kw_all = df_kw_loop[df_kw_loop[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)

        pred_kw = pred_df[pred_df[ID_COL].str.lower() == keyword.lower()].copy().sort_values(TIME_COL)
        if pred_kw.empty:
            print(f"⚠️ No predictions for keyword (h={PRED_LEN}): {keyword}")
            return

        plt.figure(figsize=(12, 4))

        plt.plot(
            kw_all[TIME_COL],
            kw_all[TARGET_COL],
            "o-",
            color="black",
            markersize=3,
            linewidth=1,
            label="Actual (full history)",
        )

        plt.plot(
            pred_kw[TIME_COL],
            pred_kw["0.5"],
            "s--",
            color="purple",
            linewidth=1.8,
            markersize=5,
            label="Forecast (median)",
        )

        plt.fill_between(
            pred_kw[TIME_COL],
            pred_kw["0.1"],
            pred_kw["0.9"],
            color="purple",
            alpha=0.25,
            label="P10–P90 interval",
        )

        plt.scatter(
            test_kw[TIME_COL],
            test_kw[TARGET_COL],
            color="red",
            marker="D",
            s=50,
            label="Test set (actual)",
            zorder=5,
        )

        plt.axvline(
            train_end,
            linestyle="--",
            color="gray",
            alpha=0.7,
            label="Train/Test split",
        )

        plt.title(f"Global Fine-Tuned Model with geo-variables | {keyword} (horizon={PRED_LEN})", fontsize=14)
        plt.xlabel("Week", fontsize=11)
        plt.ylabel("Average CPC", fontsize=11)
        plt.grid(True, linestyle="--", alpha=0.3)
        plt.legend(fontsize=9)
        plt.tight_layout()
        plt.show()

    # ============================
    # 14) PLOT YOUR SELECTED KEYWORDS (OR RANDOM FALLBACK)
    # ============================
    all_keywords_full = (
    df_kw_loop[ID_COL].dropna().astype(str).str.strip().unique().tolist())

    keywords_to_plot = get_keywords_to_plot(
        all_keywords=all_keywords_full,         # ✅ match against full dataset
        keywords_to_plot=KEYWORDS_TO_PLOT,      # ✅ your list
        n_plot_default=N_PLOT_DEFAULT,
        seed=RANDOM_SEED,
    )


    print(f"\nPlotting {len(keywords_to_plot)} keywords (global, horizon={PRED_LEN}):")
    for kw in keywords_to_plot:
        print(" -", kw)
        plot_global_forecast(kw)
